In [1]:
from scipy.signal import *
import numpy as np
import pywt
import scipy.io
import matplotlib.pyplot as plt

In [2]:
def filter_iterator(h0,h1,Ni):
    h = {}
    h[Ni] = h1
    h_ = h0
    
    for i in range(Ni-1,0,-1):
        u = upfirdn([1],h[i+1], up = 2)
        u = u[0:(len(u)-1)]
        h[i] = np.convolve(h0,u)
        u = upfirdn([1], h_, up = 2)
        u = u[0:(len(u)-1)]
        h_ = np.convolve(h0,u)
    
    h[0] = h_
        
    sam_expoentes = list(range(Ni,0,-1))
    c = sam_expoentes[0]
    sam_expoentes.insert(0,c)
    bases = [2] * (len(sam_expoentes))
    sam_factors = np.power(bases,sam_expoentes)
        
    return h,sam_factors

In [3]:
def qmf_decompostion(x,h0,h1,niveis):
    h,sampling_factors = filter_iterator(h0,h1,niveis)
    x_hat_cell = {}
    x_hat_cell[niveis] = 0
    
    N = 0
    
    for k in range(niveis, -1,-1):
        x_hat_cell[k] = np.convolve(h[k],x)
        x_hat_cell[k] = upfirdn([1], x_hat_cell[k],sampling_factors[k])
        N = N + len(x_hat_cell[k])
    
    n = 1
    
    x_hat = []
    
    for k in range(niveis):
        x_hat[n : len(x_hat_cell[k]) + n] = x_hat_cell[k]
        n += len(x_hat_cell[k])
        
    
    return x_hat,x_hat_cell
    
    
    

In [4]:
data = scipy.io.loadmat('ecg_1.mat')
y = data['x']
x = [row[0] for row in y]
fs_mat = data['fs']
fs = fs_mat[0][0]

In [6]:
wavelet = pywt.Wavelet('haar')

In [7]:
h0,h1,g0,g1 = wavelet.filter_bank

In [10]:
a,b = qmf_decompostion(x,h0,h1,3)

In [11]:
b[0]

{3: array([ 20.50609665,   0.        ,   0.        , ...,   0.        ,
        -27.57716447,   0.        ]),
 2: array([14.5,  0. ,  0. , ...,  0. ,  0. ,  0. ]),
 1: array([10.25304833,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]),
 0: array([-10.25304833,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ])}

In [7]:
def qmf_reconstruction(x_hat_cell, g0, g1, N):

    levels = len(x_hat_cell) - 1

    g,sampling_factors = filter_iterator(g0,g1,levels)

    delay_factors = np.zeros(levels+1)

    for i in range(3,levels + 1):
        delay_factors[i] = 2 * delay_factors[n - 1] + 1
    
    x = []
    x_ = []

    for i in range (1, levels + 1):
        x_ = h_hat_cell[i]
        x_ = upfirdn([1], x_, up = sampling_factors[i])
        x_ = x_[1 : len(x_) - sampling_factors[n] + 1]
    
        x = x + x_

    d = len(g0) - 1;    
    D = 2 * delay_factors(len(delay_factors)) + 1;
    D = D * d
    x = x[D + 1: len(x)]
    x = x[1 : N]
    
    return x

    

function x = qmf_reconstruction(x_hat_cell, g0, g1, N, h0, h1);
 
levels = length(x_hat_cell) - 1;
 
[g, sampling_factors] = filter_iterator(g0, g1, levels);
 
delay_factors = zeros(levels + 1, 1);
 
for n = 3 : levels + 1
    delay_factors(n) = 2 * delay_factors(n - 1) + 1;
end
 
if length(g0) == length(g1)
    d = length(g0) - 1;
else
    [t, A, d, t1, t2] = ...
    test_qmf_filters_for_perfect_reconstruction(h0, h1, g0, g1);
end
 
x = 0;
x_ = []
for n = 1 : levels + 1
    x_ = x_hat_cell{n};
    if size(x_, 1) > size(x_, 2)
        x_ = [zeros(d * delay_factors(n), 1); x_];
    else
        x_ = [zeros(1, d * delay_factors(n)) x_];
    end
    x_ = upsample(x_, sampling_factors(n));
    x_ = x_(1 : length(x_) - sampling_factors(n) + 1);
    x_ = conv(g{n}, x_);
    M = max([length(x) length(x_)]);
    if size(x_, 1) > size(x_, 2)
        x = [x; zeros(M - length(x), 1)];
        x_ = [x_; zeros(M - length(x_), 1)];
    else
        x = [x zeros(1, M - length(x))];
        x_ = [x_ zeros(1, M - length(x_))];
    end
    x = x + x_;
end
 
%z = zeros(length(g{1}) - 1, 1);
%zd = downsample(z, sampling_factors(1));
%zdu = upsample(zd, sampling_factors(1));
%zdu = zdu(1 : end - (sampling_factors(1) - 1));
%A = length(zdu);
%x = x(1 : end - A);
 
D = 2 * delay_factors(length(delay_factors)) + 1;
D = D * d;
x = x(D + 1: length(x));
x = x(1 : N);

In [12]:
b[0]

array([-10.25304833,   0.        ,   0.        , ...,   0.        ,
         0.        ,   0.        ])

In [13]:
len(b[0])

245816

In [14]:
b[1]

array([10.25304833,  0.        ,  0.        , ...,  0.        ,
        0.        ,  0.        ])

In [15]:
len(b[1])

245816

In [16]:
b[2]

array([14.5,  0. ,  0. , ...,  0. ,  0. ,  0. ])

In [17]:
b[3]

array([ 20.50609665,   0.        ,   0.        , ...,   0.        ,
       -27.57716447,   0.        ])